In [30]:
import pathlib
import json
import pandas as pd

def load_json(folder) -> pd.DataFrame:
    data = []
    for f in pathlib.Path(folder).iterdir():
        with open(f) as file:
            json_data = json.load(file)
            json_data['filename'] = f.stem  # Add filename without the .json extension
            data.append(json_data)
    # Normalize the JSON data
    normalized_data = pd.json_normalize(data)
    return pd.DataFrame(normalized_data).drop(columns = ["name"])

# Assuming the folder is 'data/training'
result = load_json('data/evaluation')
result.to_csv('training_annotate.csv', index=False)




In [31]:
import numpy as np

def get_shapes(row):
    return [(np.array(z["input"]).shape, np.array(z["output"]).shape) for z in row]

def check_equal_shape(shapes):
    return all(y[0] == y[1] for y in shapes)

result["train_shapes"] = result["train"].apply(get_shapes)
result["equal_shape"] = result["train_shapes"].apply(check_equal_shape)


In [35]:
result["equal_shape"].mean()

0.675

In [32]:
!pip install pillow

In [65]:
import numpy as np
from PIL import Image
import io
from scipy.fftpack import dct


def low_frequency_content(image, threshold=0.1):
    # Apply 2D DCT
    dct_result = dct(dct(image.T, norm='ortho').T, norm='ortho')
    
    # Calculate the total energy
    total_energy = np.sum(np.abs(dct_result)**2)
    
    # Calculate the energy in the low-frequency region
    h, w = image.shape
    low_freq_region = dct_result[:int(h*threshold), :int(w*threshold)]
    low_freq_energy = np.sum(np.abs(low_freq_region)**2)
    
    # Return the ratio of low-frequency energy to total energy
    return low_freq_energy / total_energy

def in_paint(x: np.array, y: np.array, x_test: np.array):
    assert x.shape == y.shape == x_test.shape, "All input arrays must have the same shape"
    
    # Initialize y_test with random values
    y_test = np.random.randint(0, 10, size=y.shape, dtype=np.uint8)
    
    # Concatenate arrays into a 2x2 grid
    top_row = np.concatenate([x, y], axis=1)
    bottom_row = np.concatenate([x_test, y_test], axis=1)
    full_image = np.concatenate([top_row, bottom_row], axis=0)
    
    # Calculate initial compression size
    best_size = low_frequency_content(full_image)
    
    # Hill climbing
    for _ in range(1000):  # You can adjust the number of iterations
        i, j = np.random.randint(0, y_test.shape[0]), np.random.randint(0, y_test.shape[1])
        old_value = y_test[i, j]
        
        # Try all values from 0 to 9
        for new_value in range(10):
            if new_value != old_value:
                y_test[i, j] = new_value
                bottom_row = np.concatenate([x_test, y_test], axis=1)
                full_image = np.concatenate([top_row, bottom_row], axis=0)
                new_size = low_frequency_content(full_image)
                print(new_size)
                if new_size < best_size:
                    best_size = new_size
                    break  # Keep this change and move to next pixel
        else:
            # If no better value was found, revert to the old value
            y_test[i, j] = old_value
    
    return y_test

In [62]:
sample = result[result.equal_shape == 1].iloc[0].to_dict()

In [66]:
x1 = np.array(sample["train"][0]["input"])
y1 = np.array(sample["train"][0]["output"])

x2 = np.array(sample["test"][0]["input"]) 

in_paint(x1, y1, x2)

0.4482742295837437
0.44311716816087726
0.4377647509656818
0.4389665495265482
0.43998426333772334
0.4408169538798823
0.44146415342045997
0.44192586450597543
0.4422025564992166
0.442295159221996
0.44220505379716823
0.44193406081463404
0.43235474258662776
0.42761560405606963
0.424642864129864
0.42017668473612435
0.41691146710148297
0.4118121753702306
0.4096288037527118
0.4023597419047249
0.39992348383189236
0.4011609434047411
0.40221939906889365
0.4030977852445822
0.40379552521469636
0.40431253097083497
0.40464919978433644
0.4048064075701355
0.4047854991502096
0.40458827555990784
0.3959188725482575
0.39051058299560326
0.3894006985391946
0.3873274916078206
0.3885234077649554
0.3895458490060307
0.3903937785200733
0.39106663725294605
0.39156434373503224
0.3918872906819423
0.39203633843584185
0.3920128053536509
0.3918184552846034
0.3887867803245537
0.39007350532301066
0.3911862684448315
0.3921241491036466
0.39288670574711126
0.39347397406890594
0.3938864620233519
0.3941251417346319
0.39419143

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 9, 0, 0, 0, 0],
       [0, 9, 0, 0, 9, 9, 0, 9, 0, 0],
       [0, 0, 0, 0, 9, 9, 0, 9, 9, 9]], dtype=uint8)